### Goal
Run flows using sdk and allowing for external configuration of RAG parameters. 

### Check Connections

In [ ]:
import promptflow

pf = promptflow.PFClient()

# List all the available connections
for c in pf.connections.list():
    print(c.name + " (" + c.type + ")")

acs_connection (CognitiveSearch)
aoai_connection (AzureOpenAI)
cosmodb_connection (Custom)
postgres_connection (Custom)


### Test the flow

In [ ]:
output = pf.flows.test(
    "../../rag-cosmos-postgres/flow.dag.yaml",
    inputs={
        "chat_history": [],
        "query": "What is the growth rate of Azure ML revenue in FY23Q1?",
    },
)

print(output)

[2024-04-29 21:51:37,138][promptflow.core._connection][WARNING] - Please use connection.secrets[key] to access secrets.


2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Start to run 14 nodes with concurrency level 16.
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Executing node ConfigLoader. node run id: 3abeba97-194a-49d6-bb8f-2f017ebc3051_ConfigLoader_0
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Executing node CheckExistingContext. node run id: 3abeba97-194a-49d6-bb8f-2f017ebc3051_CheckExistingContext_0
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Executing node QueryParser. node run id: 3abeba97-194a-49d6-bb8f-2f017ebc3051_QueryParser_0
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Executing node FormatConversation. node run id: 3abeba97-194a-49d6-bb8f-2f017ebc3051_FormatConversation_0
2024-04-29 21:51:39 -0700   41512 execution.flow     INFO     Node CheckExistingContext completes.
2024-04-29 21:51:39 -0700   41512 e

### Run the flow with the benchmark data file

In [ ]:
flow_path = "../../rag-cosmos-postgres/flow.dag.yaml"
data_path = "../../datasets/evalset.csv"

column_mapping = {
    "chat_history": "${data.chat_history}",
    "query": "${data.query}",
}

run_postgres_topk3 = pf.run(flow=flow_path, data=data_path, column_mapping=column_mapping)
pf.stream(run_postgres_topk3)
print(run_postgres_topk3)

[2024-04-29 21:52:10,252][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_postgres_variant_0_20240429_215210_030833, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_postgres_variant_0_20240429_215210_030833\logs.txt
[2024-04-29 21:52:35,922][promptflow._sdk._submitter.run_submitter][WARNING] - 2 out of 13 runs failed in batch run.
 Please check out C:/Users/alsavelv/.promptflow/.runs/rag_cosmos_postgres_variant_0_20240429_215210_030833 for more details.


2024-04-29 21:52:12 -0700   41512 execution.bulk     INFO     Current system's available memory is 36452.05078125MB, memory consumption of current process is 204.29296875MB, estimated available worker count is 36452.05078125/204.29296875 = 178
2024-04-29 21:52:12 -0700   41512 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 13, 'estimated_worker_count_based_on_memory_usage': 178}.
2024-04-29 21:52:16 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-2)-Process id(6992)-Line number(0) start execution.
2024-04-29 21:52:16 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-3)-Process id(39188)-Line number(1) start execution.
2024-04-29 21:52:16 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-5)-Process id(40268)-Line number(2) start execution.
2024-04-29 21:52:16 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-4)-Process id

In [ ]:
pf.get_details(run_postgres_topk3)

,inputs.chat_history,inputs.query,inputs.line_number,outputs.reply,outputs.fetched_docs,outputs.log_params
0,[],What is the quarterly revenue of Microsoft Clo...,0,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
1,[],How many Azure Arc customers does Microsoft cu...,1,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
2,[],Which companies run SAP on Azure in FY23Q1?,2,"According to the retrieved documents, in FY23Q...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
3,[],What is the annual recurring revenue of GitHub...,3,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
4,[],How many monthly active users does Power BI ha...,4,The retrieved documents do not contain informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."
5,[],What is the quarterly revenue of Microsoft Clo...,5,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
6,[],How many Azure Arc customers does Microsoft cu...,6,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
7,[],What are some examples of companies using Azur...,7,"According to the retrieved documents, in FY23Q...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
8,[],How many developers are using GitHub Copilot a...,8,"According to the retrieved documents, as of FY...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to..."
9,[],What is the growth rate of Azure Machine Learn...,9,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to..."


In [ ]:
ddf = pf.get_details(run_postgres_topk3)
#ddf.to_csv("checkoutput.csv")

## Evaluate 

In [ ]:
eval_flow_path = "../../evaluator/eval_aistudio_score/"
data_path = "../../datasets/evalset.csv"

eval_postgres_run_topk3 = pf.run(
    flow=eval_flow_path,
    run=run_postgres_topk3,
    data=data_path,
    column_mapping={
        "question": "${data.query}",
        "answer": "${data.answer}",
        "reply": "${run.outputs.reply}",
        "context": "${run.outputs.fetched_docs}",
        "log_params": "${run.outputs.log_params}"
    },
    display_name="eval_postgres_aistudio_score_topk3",
)
pf.stream(eval_postgres_run_topk3)
print(eval_postgres_run_topk3)

[2024-04-29 21:53:16,290][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240429_215315_992745, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240429_215315_992745\logs.txt


2024-04-29 21:53:20 -0700   41512 execution.bulk     INFO     Current system's available memory is 37981.23828125MB, memory consumption of current process is 214.87890625MB, estimated available worker count is 37981.23828125/214.87890625 = 176
2024-04-29 21:53:20 -0700   41512 execution.bulk     INFO     Set process count to 4 by taking the minimum value among the factors of {'default_worker_count': 4, 'row_count': 11, 'estimated_worker_count_based_on_memory_usage': 176}.
2024-04-29 21:53:24 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-7)-Process id(17156)-Line number(0) start execution.
2024-04-29 21:53:24 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-8)-Process id(6008)-Line number(1) start execution.
2024-04-29 21:53:24 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-9)-Process id(41900)-Line number(2) start execution.
2024-04-29 21:53:24 -0700   41512 execution.bulk     INFO     Process name(SpawnProcess-10)-Process i

In [ ]:
pf.get_details(eval_postgres_run_topk3)

,inputs.question,inputs.answer,inputs.reply,inputs.context,inputs.log_params,inputs.line_number,outputs.relevance,outputs.coherence,outputs.fluency,outputs.groundedness,outputs.gpt_similarity
0,What is the quarterly revenue of Microsoft Clo...,"$25 billion in quarterly revenue, up 24 percen...",The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",0,1,5,4,1,1
1,How many Azure Arc customers does Microsoft cu...,"More than 8,500 Arc customers, more than doubl...",The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",1,4,5,3,5,2
2,Which companies run SAP on Azure in FY23Q1?,"Companies like Chobani, Munich RE, Sodexo, Vol...","According to the retrieved documents, in FY23Q...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",2,5,5,5,5,5
3,What is the annual recurring revenue of GitHub...,GitHub is now at $1 billion in annual recurrin...,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",3,1,4,4,1,1
4,How many monthly active users does Power BI ha...,Power BI is the market leader in business inte...,The retrieved documents do not contain informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",4,1,5,4,1,1
5,What is the quarterly revenue of Microsoft Clo...,"$27 billion in quarterly revenue, up 22 percen...",The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",5,1,5,4,1,1
6,How many Azure Arc customers does Microsoft cu...,"More than 12,000 Arc customers, double the num...",The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",6,2,5,3,1,1
7,What are some examples of companies using Azur...,"Over 200 customers, including KPMG and Al Jaze...","According to the retrieved documents, in FY23Q...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",7,2,5,5,1,2
8,How many developers are using GitHub Copilot a...,More than one million people have used GitHub ...,"According to the retrieved documents, as of FY...","{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '2'}, 'to...",8,2,5,5,3,2
9,What is the growth rate of Azure Machine Learn...,Azure ML revenue alone has increased more than...,The retrieved documents do not provide informa...,"{""retrieved_documents"": [{""[doc0]"": {""ticker"":...","{'filter': {'Year': '23', 'Quarter': '1'}, 'to...",9,1,4,4,1,1


In [ ]:
pf.get_metrics(eval_postgres_run_topk3)

{'relevance': 1.9090909090909092,
 'coherence': 4.7272727272727275,
 'fluency': 4.090909090909091,
 'groundedness': 1.9090909090909092,
 'gpt_similarity': 1.6363636363636365}

In [ ]:
pf.visualize(eval_postgres_run_topk3)

The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-6m5glrv4.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.


### Multiple runs 
Using config.yaml file

In [ ]:
import yaml
import promptflow

pf = promptflow.PFClient()

with open("./runs_config.yaml", "r") as file:
    runs_config = yaml.safe_load(file)

# Access the contents of the config.yaml file
print(runs_config)

{'Run1': {'maxTokens': 3500, 'topK': 3, 'searchType': 'vector', 'tableName': 'msft_transcript'}, 'Run2': {'maxTokens': 3500, 'topK': 3, 'searchType': 'hybrid', 'tableName': 'msft_transcript'}}


In [ ]:
# function to update param_config systematically 
import os 
def update_param_config(flow_path, config):
    base_dir = os.path.dirname(flow_path)
    param_config_path = os.path.join(base_dir, 'param_config.yaml')

    with open(param_config_path, 'r') as file:
        param_dict = yaml.safe_load(file)

    param_dict.update(config)

    with open(param_config_path, 'w') as file:
        yaml.dump(param_dict, file)


In [ ]:
from datetime import datetime

run_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


def run_dags(
    flow_path="../../rag-cosmos-postgres/flow.dag.yaml",
    data_path="../../datasets/evalset.csv",
    eval_flow_path="../../evaluator/eval_aistudio_score/",
    config=None,
):
    column_mapping = {
        "chat_history": "${data.chat_history}",
        "query": "${data.query}",
    }
    update_param_config(flow_path, config)

    copilot_run_from_config = pf.run(
        flow=flow_path,
        data=data_path,
        column_mapping=column_mapping,
        display_name="rag_postgres_run_from_config_" + run_time,
    )


    # pf.stream(current_run)
    # print(run_topk3)

    eval_run_from_config = pf.run(
        flow=eval_flow_path,
        run=copilot_run_from_config,
        data=data_path,
        column_mapping={
            "question": "${data.query}",
            "answer": "${data.answer}",
            "reply": "${run.outputs.reply}",
            "context": "${run.outputs.fetched_docs}",
            "log_params": "${run.outputs.log_params}",
        },
        display_name="eval_run_postgres_aistudio_scores_from_config_" + run_time,
    )
    return copilot_run_from_config, eval_run_from_config

In [ ]:
i = 0
run_logs = {}
for run in runs_config:
    i += 1
    print(runs_config[run])
    copilot_run_from_config, eval_run_from_config = run_dags(config=runs_config[run])
    run_logs[i] = eval_run_from_config

{'maxTokens': 3500, 'topK': 3, 'searchType': 'vector', 'tableName': 'msft_transcript'}


[2024-04-29 21:57:34,444][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_postgres_variant_0_20240429_215734_153002, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_postgres_variant_0_20240429_215734_153002\logs.txt
[2024-04-29 21:57:59,983][promptflow._sdk._submitter.run_submitter][WARNING] - 2 out of 13 runs failed in batch run.
 Please check out C:/Users/alsavelv/.promptflow/.runs/rag_cosmos_postgres_variant_0_20240429_215734_153002 for more details.
[2024-04-29 21:58:01,007][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240429_215800_697815, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240429_215800_697815\logs.txt


{'maxTokens': 3500, 'topK': 3, 'searchType': 'hybrid', 'tableName': 'msft_transcript'}


[2024-04-29 22:00:46,830][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run rag_cosmos_postgres_variant_0_20240429_220046_075300, log path: C:\Users\alsavelv\.promptflow\.runs\rag_cosmos_postgres_variant_0_20240429_220046_075300\logs.txt
[2024-04-29 22:01:21,318][promptflow._sdk._submitter.run_submitter][WARNING] - 2 out of 13 runs failed in batch run.
 Please check out C:/Users/alsavelv/.promptflow/.runs/rag_cosmos_postgres_variant_0_20240429_220046_075300 for more details.
[2024-04-29 22:01:21,891][promptflow._sdk._submitter.run_submitter][INFO] - Submitting run eval_aistudio_score_variant_0_20240429_220121_603177, log path: C:\Users\alsavelv\.promptflow\.runs\eval_aistudio_score_variant_0_20240429_220121_603177\logs.txt


In [ ]:
for run_id in run_logs:
    print(run_id)
    pf.visualize(run_logs[run_id])

1
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-iyluiij5.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
2
The HTML file is generated at 'C:\\Users\\alsavelv\\AppData\\Local\\Temp\\pf-visualize-detail-m064il1h.html'.
Trying to view the result in a web browser...
Successfully visualized from the web browser.
